In [1]:
!pip install pycocotools

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import requests
from zipfile import ZipFile

def download_file(url, save_path):
    """Download file from the given URL and save it to the specified path."""
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(save_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# List of COCO dataset URLs
urls = [
    "http://images.cocodataset.org/zips/train2017.zip",  # Train images
    "http://images.cocodataset.org/zips/val2017.zip",    # Validation images
    "http://images.cocodataset.org/zips/test2017.zip",   # Test images
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"  # Annotations
]

# Process each URL
for url in urls:
    file_name = url.split("/")[-1]
    extract_dir = file_name.replace(".zip", "")
    print(f"Processing {file_name}...")

    # Step 1: Download the file
    print(f"Downloading {url}...")
    download_file(url, file_name)
    print(f"Downloaded {file_name}")

    # Step 2: Extract the file
    print(f"Extracting {file_name}...")
    with ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted to {extract_dir}")

    # Step 3: Clean up the downloaded zip file
    os.remove(file_name)
    print(f"Removed {file_name}")

print("All files have been downloaded and extracted successfully.")


Processing train2017.zip...
Downloaded train2017.zip
Extracting train2017.zip...
Extracted to train2017
Removed train2017.zip
Processing val2017.zip...
Downloaded val2017.zip
Extracting val2017.zip...
Extracted to val2017
Removed val2017.zip
Processing test2017.zip...
Downloaded test2017.zip
Extracting test2017.zip...
Extracted to test2017
Removed test2017.zip
Processing annotations_trainval2017.zip...
Downloaded annotations_trainval2017.zip
Extracting annotations_trainval2017.zip...
Extracted to annotations_trainval2017
Removed annotations_trainval2017.zip
All files have been downloaded and extracted successfully.


In [3]:
!ls

AttentionDistillation.ipynb  checkpoints      dense.ipynb
Bottleneck.ipynb	     checkpoints_1    id_rsa
ResNet.ipynb		     checkpoints_2    id_rsa.pub
SimpleProjection.ipynb	     checkpoints_old  ondemand
TinyImageNet.ipynb	     checkpoints_vsm  test2017
UnifiedStudentModel.ipynb    coco.ipynb       train2017
annotations_trainval2017     coco.zip	      untitled.ipynb
authorized_keys		     data	      val2017


In [13]:
import os
import torch
from pycocotools.coco import COCO
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader

# Custom dataset for COCO classification
class COCOClassification(Dataset):
    def __init__(self, root, annotation_file, transform=None):
        self.root = root
        self.coco = COCO(annotation_file)
        self.transform = transform
        self.image_ids = list(self.coco.imgToAnns.keys())
        self.classes = list(self.coco.cats.keys())

    def __len__(self):
        return len(self.image_ids)

    # Multi-label classification
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        annotations = self.coco.loadAnns(self.coco.getAnnIds(imgIds=image_id))

        # Load image
        image_info = self.coco.loadImgs(image_id)[0]
        image_path = os.path.join(self.root, image_info['file_name'])
        image = Image.open(image_path).convert("RGB")

        # Create multi-label vector
        labels = [0] * len(self.classes)
        for ann in annotations:
            category_idx = self.classes.index(ann['category_id'])
            labels[category_idx] = 1

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(labels, dtype=torch.float32)

# Initialize COCO classification data loaders
def init_coco_classification(data_dir, annotation_file, batch_size=32):
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    dataset = COCOClassification(root=data_dir, annotation_file=annotation_file, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    return data_loader

# Main script
if __name__ == "__main__":
    # Paths to the data directories and annotation files
    data_dir = "/home/yx3493/train2017/train2017"  # Path to the extracted training images
    annotation_file = "/home/yx3493/annotations_trainval2017/annotations/instances_train2017.json"  # Path to the annotation file

    print("Initializing COCO classification data loader...")
    data_loader = init_coco_classification(data_dir, annotation_file, batch_size=32)

    # Check the first batch
    for images, labels in data_loader:
        print(f"Batch of images: {images.shape}")
        print(f"Batch of labels: {labels.shape}")
        break



Initializing COCO classification data loader...
loading annotations into memory...
Done (t=22.82s)
creating index...
index created!
Batch of images: torch.Size([32, 3, 224, 224])
Batch of labels: torch.Size([32, 80])


In [11]:
annotation_file = "/home/yx3493/annotations_trainval2017/annotations/instances_train2017.json"
coco = COCO(annotation_file)
image_ids = coco.getImgIds()
for image_id in image_ids[:10]:
    image_info = coco.loadImgs(image_id)[0]
    print(image_info['file_name'])  # Should match the files in 'train2017'


loading annotations into memory...
Done (t=17.67s)
creating index...
index created!
000000391895.jpg
000000522418.jpg
000000184613.jpg
000000318219.jpg
000000554625.jpg
000000574769.jpg
000000060623.jpg
000000309022.jpg
000000005802.jpg
000000222564.jpg


In [ ]:
import os
import torch
from pycocotools.coco import COCO
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader

# Custom dataset for COCO classification
class COCOClassification(Dataset):
    def __init__(self, root, annotation_file, transform=None):
        self.root = root
        self.coco = COCO(annotation_file)
        self.transform = transform
        self.image_ids = list(self.coco.imgToAnns.keys())
        self.classes = list(self.coco.cats.keys())

    def __len__(self):
        return len(self.image_ids)

    # Multi-label classification
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        annotations = self.coco.loadAnns(self.coco.getAnnIds(imgIds=image_id))

        # Load image
        image_info = self.coco.loadImgs(image_id)[0]
        image_path = os.path.join(self.root, image_info['file_name'])
        if not os.path.exists(image_path):
            print(f"Warning: File {image_path} not found. Skipping.")
            return None

        image = Image.open(image_path).convert("RGB")

        # Create multi-label vector
        labels = [0] * len(self.classes)
        for ann in annotations:
            category_idx = self.classes.index(ann['category_id'])
            labels[category_idx] = 1

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(labels, dtype=torch.float32)

# Initialize COCO classification data loaders
def init_coco_classification(data_dir, annotation_file, batch_size=32):
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    dataset = COCOClassification(root=data_dir, annotation_file=annotation_file, transform=transform)

    # Filter out None entries caused by missing files
    valid_data = [entry for entry in map(dataset.__getitem__, range(len(dataset))) if entry is not None]
    valid_dataset = torch.utils.data.TensorDataset(
        torch.stack([entry[0] for entry in valid_data]),
        torch.stack([entry[1] for entry in valid_data])
    )

    data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

    return data_loader

# Main script
if __name__ == "__main__":
    # Paths to the data directories and annotation files
    data_dir = "/home/yx3493/train2017"  # Path to the extracted training images
    annotation_file = "/home/yx3493/annotations_trainval2017/annotations/instances_train2017.json"  # Path to the annotation file

    print("Initializing COCO classification data loader...")
    data_loader = init_coco_classification(data_dir, annotation_file, batch_size=32)

    # Check the first batch
    for images, labels in data_loader:
        print(f"Batch of images: {images.shape}")
        print(f"Batch of labels: {labels.shape}")
        break


In [5]:
!pwd

/home/yx3493
